In [3]:
search_for = 125
start_from = 0
threads = 10
thread = 4
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.0022373199462890625
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 35
all cases: 320327931
type: [1, 1, 1, 1, 125] 125
cases of this type: 244140625
100000 13.84418421815459
200000 22.79756134346277
300000 25.546076556800177
400000 25.323577998492485
500000 25.79522886677417
600000 25.520105106463117
700000 25.4535432631466
800000 26.529583255718556
900000 27.044580428687073
1000000 13.963305486506298
1100000 25.302213926226027
1200000 25.137822017139193
1300000 24.53223459007753
1400000 24.21418828076168
1500000 26.641403131061768
1600000 24.72413847742435
1700000 24.1672881188012
1800000 25.125999435021566
1900000 25.287348310392712
2000000 7.937429106430756
2100000 26.279831418702106
2200000 28.402865356837424
2300000 28.17140289742066
2400000 28.09363797917616
2500000 27.480621944662346
2600000 27.543513242817333
2700000 25.912425849978877
2800000 27.468177552079325
2900000 25.34340369509248
3000000 12.

29400000 23.82315945750555
29500000 24.29877288350093
29600000 9.868384789400098
29700000 23.68151628836917
29800000 22.77554803254206
29900000 22.869223743881797
30000000 21.76817278685851
30100000 21.144890080102122
30200000 20.108638605583355
30300000 9.80971038246768
30400000 20.932806685756486
30500000 21.73367627646164
30600000 21.29936514154193
30700000 21.99601222788798
30800000 23.041198893749513
30900000 23.419546192284287
31000000 17.802001913923974
31100000 16.815202114285277
31200000 23.30548400372424
31300000 23.56740442459036
31400000 23.243880171326307
31500000 18.598405472065988
31600000 16.14572829851824
31700000 23.78504362645458
31800000 22.93979585299043
31900000 22.87040171031685
32000000 23.287427178419772
32100000 20.690966857765495
32200000 19.14749833184153
32300000 10.382136435649015
32400000 20.671290964489828
32500000 21.021176046649675
32600000 22.177558647452773
32700000 22.01854279978294
32800000 23.964080899815745
32900000 22.391003711468255
33000000 10

59100000 10.801391336531607
59200000 17.566607041174482
59300000 19.537733088031388
59400000 18.358334760937726
59500000 19.98283645694279
59600000 8.487028747910495
59700000 18.768148168938065
59800000 18.99649357287138
59900000 18.028283927615867
60000000 19.243668823937423
60100000 9.984590685437587
60200000 19.496654146890275
60300000 20.734168908428625
60400000 20.101600059060704
60500000 20.720417654817968
60600000 22.22762382892717
60700000 20.62702421380955
60800000 20.575898667480484
60900000 19.433158618476956
61000000 20.59571066571402
61100000 8.512350017570382
61200000 18.68773720769991
61300000 18.466032311388727
61400000 18.260018764679824
61500000 16.2668136240852
61600000 9.559824363821047
61700000 18.10496851158936
61800000 18.435916018151776
61900000 18.459332110534802
62000000 14.72115267106979
62100000 13.410407269069852
62200000 20.59029068852121
62300000 20.088042296096216
62400000 20.05477106412883
62500000 21.04087218224924
62600000 21.349645403119087
62700000 

88800000 15.687300039070607
88900000 6.202249240174684
89000000 17.619390288005906
89100000 16.025839810303097
89200000 9.387259768682956
89300000 15.798242310127446
89400000 16.178201731422096
89500000 17.592444581408607
89600000 17.161344960939182
89700000 17.53870995749212
89800000 17.491457336465782
89900000 18.631951896968438
90000000 18.569363797998424
90100000 17.989591798080003
90200000 17.468277095751162
90300000 17.049807277977276
90400000 16.604433321543528
90500000 16.051461412567363
90600000 8.79132291833278
90700000 16.04375477070192
90800000 14.006408193405761
90900000 10.08349372639457
91000000 16.720655956670633
91100000 8.998786136323098
91200000 16.05494112643881
91300000 17.071320117797686
91400000 15.718555681772031
91500000 16.506894118459908
91600000 17.882366941216564
91700000 18.110746278037272
91800000 17.45998884800218
91900000 17.697783315367044
92000000 17.904982314787443
92100000 16.724378338710146
92200000 15.922023634270783
92300000 17.0013867680241
9240

117800000 1.644265858181161
117900000 1.5541766605677982
118000000 1.7099886197278453
118100000 1.586239942894727
118200000 1.6048388058162155
118300000 1.5964275907759546
118400000 1.6606206420448189
118500000 1.6793780953702788
118600000 1.6208946468617147
118700000 2.0614403511994257
118800000 2.0760254124867816
118900000 1.5838743186165298
119000000 1.6387336989568013
119100000 1.5558115937290906
119200000 1.5294315942879364
119300000 1.5023878154034211
119400000 1.515036949335722
119500000 1.5187113999124089
119600000 1.5831872898744597
119700000 1.5161735868956185
119800000 1.504263099921688
119900000 1.6032322360060642
120000000 1.6305217685477973
120100000 1.538096518288421
120200000 1.5082595191218913
120300000 1.587968277448505
120400000 1.6033656662606788
120500000 1.5896723762001124
120600000 1.7201948278015435
120700000 1.6709007467634742
120800000 1.622770732645091
120900000 1.5618561573004484
121000000 1.6689446929382044
121100000 1.7000999980359912
121200000 1.815959969

...

204100000 5.526801392277762
204200000 5.5671683089089825
204300000 7.90739153306365
204400000 7.851018294380115
204500000 8.716898196151321
204600000 8.453521095935034
204700000 8.713340995350679
204800000 3.85476561791774
204900000 8.979058795896803
205000000 8.705327118926215
205100000 8.733095429491296
205200000 8.563296683527248
205300000 8.265199808871708
205400000 5.165817220645168
205500000 7.635633844988417
205600000 8.043617849808928
205700000 7.94706383828291
205800000 7.988822011773487
205900000 7.617946599252752
206000000 7.560673904907083
206100000 2.959412690414329
206200000 7.843208332471962
206300000 7.569684194992165
206400000 7.778620908241064
206500000 8.039551807988964
206600000 8.181497597424588
206700000 8.123415020402858
206800000 4.435705688040467
206900000 8.400950746467213
207000000 8.51547440107463
207100000 8.672754013396876
207200000 8.595199211404871
207300000 7.5765272808007476
207400000 5.305305234540085
207500000 8.794226168872271
207600000 8.1372096822

233400000 4.225684843715107
233500000 4.489318518585632
233600000 6.575793660868727
233700000 6.34189138715612
233800000 6.229142762184572
233900000 6.512694198998436
234000000 6.730228220579097
234100000 6.580543573058363
234200000 6.585067800536787
234300000 4.280206192542184
234400000 5.526034377089062
234500000 3.2207862323099516
234600000 6.47274841753536
234700000 6.356046542736499
234800000 6.518493612956031
234900000 6.446092623487161
235000000 6.189139828960055
235100000 6.198311798419463
235200000 6.45299656224252
235300000 6.370560519415982
235400000 2.25706475606141
235500000 6.512358772648684
235600000 6.419492196455249
235700000 6.2958601446888895
235800000 6.445521833563709
235900000 6.63792975768148
236000000 6.59616297623004
236100000 6.468204960241968
236200000 6.598766134763671
236300000 2.776906587791627
236400000 4.102059341705388
236500000 5.426841224593428
236600000 6.327052684506875
236700000 6.197652784966141
236800000 6.357872150564504
236900000 6.219142685672

262000000 1.6142280881521818
262100000 1.7309020621316362
262200000 1.7465354950809717
262300000 1.630440122142701
262400000 1.5508718350683817
262500000 1.972219302237793
262600000 1.0760753745576852
262700000 0.48540803431577345
262800000 0.4835588336826265
262900000 1.0733406180403404
263000000 2.1596707517771163
263100000 1.5516723179456233
263200000 1.667295108794291
263300000 1.7221453157999536
263400000 1.5627823561388632
263500000 1.6771561856716766
263600000 1.6915269940202255
263700000 1.908484955442623
263800000 1.7726865117266437
263900000 1.6904456344748255
264000000 1.6095577117114126
264100000 1.5747458369120249
264200000 1.5211001684638157
264300000 1.6012060973139008
264400000 1.7357777175256404
264500000 1.3928775390395158
264600000 0.7544292466375272
264700000 0.4709980847382307
264800000 0.6571620448624504
264900000 1.4302981828582764
265000000 1.8743668102682967
265100000 1.583724812697872
265200000 1.4762227423882208
265300000 1.5611264178586914
265400000 1.580225

290400000 0.9718997468161603
290500000 0.852973892296885
290600000 0.8438879634362757
290700000 0.8847046324407956
290800000 0.9185856154028236
290900000 0.8677546514785481
291000000 0.9565728316720604
291100000 0.9905617872861703
291200000 0.9636970334154288
291300000 0.8318623368442952
291400000 0.8679282571364363
291500000 0.9075154796630064
291600000 0.8057442870811441
291700000 0.75154018488768
291800000 0.9372498642346065
291900000 0.5314536035228015
292000000 0.23782360784951315
292100000 0.24075129798961084
292200000 0.5313473293762206
292300000 0.9561390245351274
292400000 0.720140764405785
292500000 0.8584609633241057
292600000 0.8808953103023482
292700000 0.7884159218009107
292800000 0.8695592320774556
292900000 0.8409611480730204
best so far: 0
type: [1, 1, 1, 25, 5] 125
cases of this type: 9765625
293000000 0.9245282744602468
293100000 0.8187267785900573
293200000 0.7475818267672306
293300000 0.7875011220022924
293400000 0.7622278135261098
293500000 0.6518161271560239
2936

316900000 0.019574570492942464
317000000 0.02240317167096734
317100000 0.019626954365712407
best so far: 0
type: [1, 1, 25, 5, 1] 125
cases of this type: 78125
317200000 0.01876717865102821
best so far: 0
type: [1, 1, 125, 1, 1] 125
cases of this type: 15625
best so far: 0
type: [1, 5, 1, 1, 25] 125
cases of this type: 1953125
317300000 0.027446320195838474
317400000 0.05688597507252097
317500000 0.05094900292492045
317600000 0.050046800213032955
317700000 0.048307091276623804
317800000 0.037038121309245955
317900000 0.03217737930598921
318000000 0.04963745993795991
318100000 0.03412915491736796
318200000 0.015730640801839033
318300000 0.012880514053308964
318400000 0.013224121518128448
318500000 0.019149081524579396
318600000 0.033497987533388535
318700000 0.03163907254550457
318800000 0.024641427315025194
318900000 0.025574530306595566
319000000 0.016566968833400145
319100000 0.017878395734972425
319200000 0.017246036650643747
best so far: 0
type: [1, 5, 1, 5, 5] 125
cases of this ty